In [ ]:
# I ran this notebook on Kaggle. It can be accessed at: 
# https://www.kaggle.com/abdelrhmanghreeb/classification-using-pca-on-pima-data-set

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

pima-indians-diabetes.csv



In [4]:
from sklearn import cross_validation
from sklearn.metrics import accuracy_score

In [5]:
dataset = np.loadtxt('../input/pima-indians-diabetes.csv', delimiter=",")
#split data into X (the 8 features) and Y (the label)
X = dataset[:,0:8]
Y = dataset[:,8]
X.shape

(768, 8)

In [6]:
#subtract the mean from your data: X = X - X.mean(axis=1).reshape(-1, 1)
xAdjusted = X- X.mean(axis=1, keepdims=True) #This is done inside the covar calculations
X = X.T

In [7]:
def calcCovMatrixEntry(xx, yy):
    xx = xx-xx.mean(axis=0, keepdims=True)
    yy = yy-yy.mean(axis=0, keepdims=True)
    return ((np.sum(xx*yy))/(xx.shape[0]-1)) #cov(xx,yy)
def calcCovMatrix(xxx):
    cm = np.empty([xxx.shape[0],xxx.shape[0]])

    for i in range(xxx.shape[0]):
        for j in range(xxx.shape[0]):
            cm[i][j]=calcCovMatrixEntry(xxx[i],xxx[j])
    return cm

In [9]:
#calculate the covariance matrix of the 8 features MAUALLY
#xxx = np.array([[-2.1, -1,  4.3],[3,  1.1,  0.12]])
covMatrix = calcCovMatrix(X)
print (X.shape)
print (covMatrix.shape)
#calculate the eigenvectors
from numpy import linalg as LA
w, v = LA.eig(covMatrix) #w= eigenvalues, v=eigenvectors
print ('w= ', len(w))
print ('v= ', v.shape)

(8, 768)
(8, 8)
w=  8
v=  (8, 8)


In [11]:
#order the eigenvectors by eigenvalues
w = np.array(w)
sortedV = np.empty(v.shape)
j =0

for i in w.argsort()[::-1]:
    sortedV[j] =v[i]
    j += 1
print (sortedV)

[[ -2.02176587e-03   2.26488861e-02  -2.24649003e-02  -4.90459604e-02
   -1.51612874e-01  -5.04730888e-03   9.86672995e-01  -6.10123250e-03]
 [  9.78115765e-02   9.72210040e-01   1.43428710e-01   1.19830016e-01
    8.79407680e-02   5.07391813e-02   8.83426114e-04   8.25459539e-04]
 [  1.60930503e-02   1.41909330e-01  -9.22467192e-01  -2.62742788e-01
    2.32165009e-01   7.56365525e-02  -1.22975947e-03  -5.20865450e-04]
 [  6.07566861e-02  -5.78614699e-02  -3.07013055e-01   8.84369380e-01
   -2.59973487e-01   2.21363068e-01  -3.76444746e-04   2.54871909e-03]
 [  9.93110844e-01  -9.46266913e-02   2.09773019e-02  -6.55503615e-02
    1.72312241e-04  -6.13326472e-03   1.42307394e-03   2.68965921e-04]
 [  1.40108085e-02   4.69729766e-02  -1.32444542e-01   1.92801728e-01
   -2.14744823e-02  -9.70776708e-01  -2.73046214e-03   2.67341863e-03]
 [  5.37167919e-04   8.16804621e-04  -6.39983017e-04   2.69908637e-03
   -1.64080684e-03  -2.02903702e-03  -6.34402965e-03  -9.99972146e-01]
 [ -3.5647443

In [12]:
#construct the feature vector
def getReducedData(number_of_eigenvectors_to_consider):
    feature_vector = sortedV[:number_of_eigenvectors_to_consider]

    #projection: deriving the new dataset with reduced dimensionality
    reducedData = np.dot(feature_vector,xAdjusted.T)
    return reducedData

In [15]:
import xgboost
model = xgboost.XGBClassifier() #use the gradient boosted decision trees
model.fit(X_train,y_train)

from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_train, y_train, cv=5) #use 5 folds
print(scores)

[ 0.71223022  0.71942446  0.70289855  0.76086957  0.7080292 ]


In [16]:
print("Average Cross Validation Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Average Cross Validation Accuracy: 0.72 (+/- 0.04)


In [17]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 70.13%


In [24]:
def classify_using_PCA(num_of_eigenvectors_to_use):
    X_reduced = getReducedData(num_of_eigenvectors_to_use)
    print (X_reduced.shape)
    print (Y.shape)
    #split data (X, Y) into train and test sets 
    seed = 7 #A random permutation, to split the data randomly
    test_size = 0.1
    X_train , X_test, y_train, y_test= cross_validation.train_test_split(X_reduced.T,Y,test_size=test_size,random_state=seed)
    model = xgboost.XGBClassifier() #use the gradient boosted decision trees
    model.fit(X_train,y_train)
    
    scores = cross_val_score(model, X_train, y_train, cv=5) #use 5 folds
    return("Used Eigen Vectors: %d Average Cross Validation Accuracy: %0.2f (+/- %0.2f)" % (num_of_eigenvectors_to_use, scores.mean(), scores.std() * 2))

In [25]:
for i in range(8):
    print (classify_using_PCA(i+1))
   

(1, 768)
(768,)
Used Eigen Vectors: 1 Average Cross Validation Accuracy: 0.65 (+/- 0.03)
(2, 768)
(768,)
Used Eigen Vectors: 2 Average Cross Validation Accuracy: 0.72 (+/- 0.04)
(3, 768)
(768,)
Used Eigen Vectors: 3 Average Cross Validation Accuracy: 0.73 (+/- 0.03)
(4, 768)
(768,)
Used Eigen Vectors: 4 Average Cross Validation Accuracy: 0.73 (+/- 0.03)
(5, 768)
(768,)
Used Eigen Vectors: 5 Average Cross Validation Accuracy: 0.73 (+/- 0.03)
(6, 768)
(768,)
Used Eigen Vectors: 6 Average Cross Validation Accuracy: 0.74 (+/- 0.04)
(7, 768)
(768,)
Used Eigen Vectors: 7 Average Cross Validation Accuracy: 0.74 (+/- 0.05)
(8, 768)
(768,)
Used Eigen Vectors: 8 Average Cross Validation Accuracy: 0.74 (+/- 0.04)


In [165]:
X_reduced = getReducedData(2).T
print (X_reduced.shape)
print (X.shape)
print (Y.shape)

print np.var(X)
print np.var(X_reduced)

(768L, 2L)
(768L, 8L)
(768L,)
3407.40378064
4309.1660389


In [179]:
# calculating variance
for i in range(8):
    print ("Used Eigen Vectors: %d, Variance: %0.5f" %  (i+1,(np.sum(w[:i+1]))/(np.sum(w))))

Used Eigen Vectors: 1, Variance: 0.88855
Used Eigen Vectors: 2, Variance: 0.95014
Used Eigen Vectors: 3, Variance: 0.97593
Used Eigen Vectors: 4, Variance: 0.98901
Used Eigen Vectors: 5, Variance: 0.99645
Used Eigen Vectors: 6, Variance: 0.99948
Used Eigen Vectors: 7, Variance: 0.99999
Used Eigen Vectors: 8, Variance: 1.00000
